In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
import tensorflow.keras
import numpy as np

In [3]:
path = tensorflow.keras.utils.get_file(
'all.txt',
origin='https://raw.githubusercontent.com/ayyucedemirbas/Star-Trek-TOS-Transcripts/main/all.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 2489987


In [4]:
maxlen = 50
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 829979
Unique characters: 66
Vectorization...


In [5]:
from tensorflow.keras import layers
model = tensorflow.keras.models.Sequential([
                                 layers.LSTM(128, input_shape=(maxlen, len(chars))),
                                 layers.Dense(len(chars), activation='softmax')

])

In [6]:
optimizer = tensorflow.keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [7]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [ ]:
import random
import sys
epochs=6
for epoch in range(1, epochs):
  print('epoch', epoch)
  model.fit(x, y, batch_size=32, epochs=1)
  start_index = random.randint(0, len(text) - maxlen - 1)
  generated_text = text[start_index: start_index + maxlen]
  print('--- Generating with seed: "' + generated_text + '"')
  for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)
    for i in range(400):
      sampled = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
      preds = model.predict(sampled, verbose=0)[0]
      next_index = sample(preds, temperature)
      next_char = chars[next_index]
      generated_text += next_char
      generated_text = generated_text[1:]
      sys.stdout.write(next_char)

epoch 1
25937/25937 [==============================] - 366s 14ms/step - loss: 1.5477
--- Generating with seed: "rbiting triacus, then the men i beamed down are de"
------ temperature: 0.2
rbiting triacus, then the men i beamed down are destroy and the ship ship the ship.
spock: you should be to be commander to see the ship and the ship and the ship the ship the ship and the ship the transporter roomme. they will be this we have the captain, you are the ship are the transporter rond.
spock: i want to see the ship and they was a see the ship with a monol starfleet and the ship the ship the ship the ship the ship with the transporter------ temperature: 0.5
ip the ship the ship the ship with the transporter roommane, they same. all talk and starb's consimend and has were to seem the enterprise.
spock [oc]: captai

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


n?
spock: i'm about the espent we do not been thoust a been the under on the was the garing to be the power suches now show has beam we think you want to see it.
kirk: i think the has in the sure is inswiter to stating and the ship something. what has been at little must a poing is a command------ temperature: 1.0
 what has been at little must a poing is a commandery instante rememmyon it.
(turnseingy blilonation pertaresadly his fillon his similar of it, but iashilly to you pad to divated my tolectional.
kirk: on a deval i final fraced orders and read now well good trying are withypes the at natory. esces iuble blustate ergon.
ybe figll contact agtingtines.
nomat [on views.
, vict he's roome.
renteriton: assoe. he ondt sitts that ive hority.
stolice plane------ temperature: 1.2
ssoe. he ondt sitts that ive hority.
stolice planet]

pack survives hentt.
pike: mccoy irin, tlier how uuspars.
kirk: your command mit of wathes.s hallim aughure, captain.
spock: syaw geroinng bild.)
oride: did